<a href="https://colab.research.google.com/github/ido177/colab/blob/main/group/group.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
!mv _sqlite3.cpython-37m-x86_64-linux-gnu.so /usr/lib/python3.7/lib-dynload/
import os
os.kill(os.getpid(), 9)

Downloading...
From: https://drive.google.com/uc?id=1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
To: /content/_sqlite3.cpython-37m-x86_64-linux-gnu.so
6.33MB [00:00, 99.9MB/s]


# Загрузка БД

In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
sqlite3.sqlite_version

'3.36.0'

In [3]:
con = sqlite3.connect('db')

In [4]:
!gdown --id 1Gj0qVU0Acv-HqRjTIEwvkO4TqJYijehR

Downloading...
From: https://drive.google.com/uc?id=1Gj0qVU0Acv-HqRjTIEwvkO4TqJYijehR
To: /content/german_credit_augmented.csv
100% 69.6k/69.6k [00:00<00:00, 25.0MB/s]


In [5]:
df = pd.read_csv('/content/german_credit_augmented.csv')

In [6]:
df.head()

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,NaN,NaN,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,NaN,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,NaN,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,NaN,776,12,radio/TV,0,2007-07-21 13:22:14,358


head выводит 5 первых значений, tail - 5 последних


In [7]:
df['contract_dt'] = pd.to_datetime(df['contract_dt'],format='%Y-%m-%d %H:%M:%S')

to_datetime меняет формат данных в дату
% помогает указать переменную

In [8]:
df.dtypes

age                          int64
sex                         object
job                          int64
housing                     object
saving_accounts             object
checking_account            object
credit_amount                int64
duration                     int64
purpose                     object
default                      int64
contract_dt         datetime64[ns]
client_id                    int64
dtype: object

возможные типы данных

In [9]:
df.to_sql('german_credit',con,index=False,if_exists='replace')

Заливаем таблицу в БД
Название, выбор бд, наличие индекса, замена существующих

In [10]:
def select(sql):
  return pd.read_sql(sql,con)

# Сводная таблица

функция селекта

In [ ]:
sql = ''' select t.sex,
count(*) as cnt,
avg(t.credit_amount) as credit_amount_avg


from german_credit t
group by t.sex'''

In [ ]:
select(sql)

,sex,cnt,credit_amount_avg
0,female,310,2877.774194
1,male,690,3448.040580


In [ ]:
df.groupby('sex')['credit_amount'].agg(['count','mean'])

,count,mean
sex,,
female,310,2877.774194
male,690,3448.040580


GROUP BY позволяет объединять значения нескольких колонок и подводить их к значению одной из колонок. В нашем случае мы берем колонку секс(2 строки) и берем еще колонку, в которой куча строк. Применяем обобщающее значение

In [ ]:
sql = '''select count(distinct t.housing), count(t.housing) from german_credit t '''

In [1]:
select(sql)

NameError: ignored

In [15]:
df.groupby('checking_account')['credit_amount'].count()

checking_account
little      274
moderate    269
rich         63
Name: credit_amount, dtype: int64

In [14]:
df.groupby('checking_account',dropna=False)['credit_amount'].count()

checking_account
little      274
moderate    269
rich         63
NaN         394
Name: credit_amount, dtype: int64

In [25]:
sql = ''' select

sum(case when t.checking_account is null then 1 else 0 end) as is_null, 
count(case when t.checking_account is null then 1 else 0 end) as is_null2

from german_credit t'''

In [26]:
select(sql)

,is_null,is_null2
0,394,1000


In [27]:
t = pd.DataFrame({'col1': [1,np.nan,2]})

In [28]:
t

,col1
0,1.0
1,NaN
2,2.0


In [29]:
t.to_sql('null_test',con,index=False,if_exists='replace')

In [32]:
sql = ''' select count(t.col1), count(*), count(1) from null_test t'''

In [33]:
select(sql)

,count(t.col1),count(*),count(1)
0,2,3,3


In [34]:
sql = '''select 

t.checking_account,

coalesce(t.checking_account, 'no_info')

from german_credit t'''

coalesce заменяет значение пропуска

In [35]:
select(sql)

,checking_account,"coalesce(t.checking_account, 'no_info')"
0,None,no_info
1,little,little
2,None,no_info
3,None,no_info
4,None,no_info
...,...,...
995,little,little
996,moderate,moderate
997,moderate,moderate
998,None,no_info


In [52]:
t = pd.DataFrame({'id':[1,2,1],
                  'name':['a','b','a']})

In [37]:
t

,id,name
0,1,a
1,2,b
2,3,c


In [53]:
t.to_sql('dupl_test',con,index=False,if_exists='replace')

In [50]:
sql = '''select t.id, t.name, count(*) as cnt from dupl_test t
group by t.id, t.name'''

In [54]:
select(sql)

,id,name,cnt
0,1,a,2
1,2,b,1


In [56]:
sql = '''select t.id, t.name, count(*) as cnt from dupl_test t

group by t.id, t.name

having count(*) > 1
'''

In [57]:
select(sql)

,id,name,cnt
0,1,a,2


In [58]:
t = pd.DataFrame({'id':[1,1,2,2,3],
                  'name':['a','b','c','d','e']})

In [59]:
t.to_sql('dupl_test',con,index=False,if_exists='replace')

In [60]:
t

,id,name
0,1,a
1,1,b
2,2,c
3,2,d
4,3,e


Идут дубли id

In [62]:
sql = '''select t.id, count(*) as cnt from dupl_test t

group by t.id

having count(*) > 1
'''

In [64]:
select(sql)

,id,cnt
0,1,2
1,2,2


In [67]:
sql = ''' select * from dupl_test t
where t.id in (select t.id cnt from dupl_test t

group by t.id

having count(*) > 1)'''

In [69]:
select(sql)

,id,name
0,1,a
1,1,b
2,2,c
3,2,d


In [83]:
sql = ''' select t.contract_dt, date(t.contract_dt, 'start of month') as month,
count(*) as cnt,
count(distinct t.client_id) as client_id_cnt_dst,
sum(t.credit_amount) as credit_amount_sum,
avg(credit_amount) as credit_amount_avg

from german_credit t
group by month
order by month'''

In [84]:
select(sql)

,contract_dt,month,cnt,client_id_cnt_dst,credit_amount_sum,credit_amount_avg
0,2007-05-20 18:30:19,2007-05-01,81,81,207663,2563.740741
1,2007-06-19 09:54:24,2007-06-01,74,74,239594,3237.756757
2,2007-07-21 13:22:14,2007-07-01,71,71,224333,3159.619718
3,2007-08-01 18:36:24,2007-08-01,57,57,178569,3132.789474
4,2007-09-11 20:27:28,2007-09-01,58,58,186909,3222.568966
5,2007-10-03 06:52:44,2007-10-01,70,70,188534,2693.342857
6,2007-11-29 00:20:44,2007-11-01,87,87,300504,3454.068966
7,2007-12-18 13:03:24,2007-12-01,77,77,273973,3558.090909
8,2008-01-20 14:51:24,2008-01-01,93,93,288080,3097.634409
9,2008-02-14 01:58:09,2008-02-01,55,55,211128,3838.690909


In [86]:
pd.Series(range(600)).sample(1000,replace=True)

487    487
189    189
391    391
271    271
84      84
      ... 
361    361
175    175
268    268
168    168
446    446
Length: 1000, dtype: int64

In [93]:
df['client_id_2'] = pd.Series(range(600)).sample(1000,replace=True).values

In [88]:
df.to_sql('german_credit_not_uniq',con,index=False,if_exists='replace')

In [96]:
sql = ''' select date(t.contract_dt, 'start of month') as month,
count(*) as cnt,
count(distinct t.client_id_2) as client_id_cnt_dst,
sum(t.credit_amount) as credit_amount_sum,
avg(credit_amount) as credit_amount_avg

from german_credit_not_uniq t
group by month
order by month'''

In [97]:
select(sql)

,month,cnt,client_id_cnt_dst,credit_amount_sum,credit_amount_avg
0,2007-05-01,81,78,207663,2563.740741
1,2007-06-01,74,70,239594,3237.756757
2,2007-07-01,71,66,224333,3159.619718
3,2007-08-01,57,52,178569,3132.789474
4,2007-09-01,58,55,186909,3222.568966
5,2007-10-01,70,66,188534,2693.342857
6,2007-11-01,87,81,300504,3454.068966
7,2007-12-01,77,72,273973,3558.090909
8,2008-01-01,93,86,288080,3097.634409
9,2008-02-01,55,51,211128,3838.690909


In [110]:
sql = ''' select 
case 
when t.credit_amount <= 1000 then '1. < 1000' 
when t.credit_amount <= 2000 then '2. 1000-2000'
when t.credit_amount <= 3000 then '3. 2000-3000'
when t.credit_amount <= 4000 then '4. >= 3000'

else 'other' end as credit_amount_bin,

count(1)

from german_credit t
group by 1'''

1 - номер колонки, можно прописать credit_amount_bin

In [111]:
select(sql)

,credit_amount_bin,count(1)
0,1. < 1000,116
1,2. 1000-2000,316
2,3. 2000-3000,188
3,4. >= 3000,134
4,other,246


In [123]:
sql = ''' select t.housing, t.sex,
count(case when t.sex = 'female' then 1 else null end) as female,
count(case when t.sex = 'male' then 1 else null end) as male,

count(*) as cnt

from german_credit t
group by t.housing
'''

In [126]:
select(sql)

,housing,sex,female,male,cnt
0,free,male,19,89,108
1,own,male,196,517,713
2,rent,female,95,84,179


In [127]:
sql = ''' select distinct t.purpose from german_credit t'''

In [129]:
list(select(sql)['purpose'].values)

['radio/TV',
 'car',
 'education',
 'furniture/equipment',
 'repairs',
 'business',
 'domestic appliances',
 'vacation/others']

In [130]:
purpose = list(select(sql)['purpose'].values)

In [133]:
for p in purpose:
  print(f"count(case when t.purpose = '{p}' then 1 else null end) as {p.replace(' ','').replace('/','')},")

count(case when t.purpose = 'radio/TV' then 1 else null end) as radioTV,
count(case when t.purpose = 'car' then 1 else null end) as car,
count(case when t.purpose = 'education' then 1 else null end) as education,
count(case when t.purpose = 'furniture/equipment' then 1 else null end) as furnitureequipment,
count(case when t.purpose = 'repairs' then 1 else null end) as repairs,
count(case when t.purpose = 'business' then 1 else null end) as business,
count(case when t.purpose = 'domestic appliances' then 1 else null end) as domesticappliances,
count(case when t.purpose = 'vacation/others' then 1 else null end) as vacationothers,


In [136]:
sql = ''' select t.housing, t.sex,
count(case when t.purpose = 'radio/TV' then 1 else null end) as radioTV,
count(case when t.purpose = 'car' then 1 else null end) as car,
count(case when t.purpose = 'education' then 1 else null end) as education,
count(case when t.purpose = 'furniture/equipment' then 1 else null end) as furnitureequipment,
count(case when t.purpose = 'repairs' then 1 else null end) as repairs,
count(case when t.purpose = 'business' then 1 else null end) as business,
count(case when t.purpose = 'domestic appliances' then 1 else null end) as domesticappliances,
count(case when t.purpose = 'vacation/others' then 1 else null end) as vacationothers,

count(*) as cnt

from german_credit t
group by t.housing
'''

In [137]:
select(sql)

,housing,sex,radioTV,car,education,furnitureequipment,repairs,business,domesticappliances,vacationothers,cnt
0,free,male,15,55,15,11,3,5,0,4,108
1,own,male,227,219,34,122,17,76,10,8,713
2,rent,female,38,63,10,48,2,16,2,0,179


In [162]:
cat = '''select t.purpose, 
case when t.purpose like '%rep%' then 'repairs'

when t.purpose like '%car%' or t.purpose like '%auto%' then 'car'

else 'others' end as purp_cnt,

count(1)
from german_credit t
group by t.purpose
'''


In [161]:
select(sql)

,purpose,purp_cnt,count(1)
0,business,others,97
1,car,car,337
2,domestic appliances,others,12
3,education,others,59
4,furniture/equipment,others,181
5,radio/TV,others,280
6,repairs,repairs,22
7,vacation/others,others,12


In [171]:
sql = f'''select t.purpose
from ({cat}) t

group by t.purpose
order by count(*) desc
'''


In [172]:
select(sql)

,purpose
0,vacation/others
1,repairs
2,radio/TV
3,furniture/equipment
4,education
5,domestic appliances
6,car
7,business
